In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd

#from skimage.io import imread
#from skimage.transform import resize
from PIL import Image, ImageOps, ImageFilter
#from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split

import tensorflow.keras as keras

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import *
from keras.callbacks import *
from keras.preprocessing.image import *

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16

In [4]:

breed_list = os.listdir("/content/drive/MyDrive/input/images/Images/")


In [5]:
# Mapping of labels and numbers
label_maps = {}
label_maps_rev = {}
for i, v in enumerate(breed_list[:10]):
    label_maps.update({v: i})
    label_maps_rev.update({i : v})

In [7]:
def paths_and_labels():
    paths = list()
    labels = list()
    targets = list()
    for breed in breed_list[:10]:
        base_name = "/content/drive/MyDrive/data/{}/".format(breed)
        for img_name in os.listdir(base_name):
            paths.append(base_name + img_name)
            labels.append(breed)
            targets.append(label_maps[breed])
    return paths, labels, targets

paths, labels, targets = paths_and_labels()

assert len(paths) == len(labels)
assert len(paths) == len(targets)
num_classes = len(breed_list)
targets = keras.utils.to_categorical(targets, num_classes=num_classes)

In [8]:
train_data_dir = "/content/drive/MyDrive/data"
img_width = 224
img_height = 224
batch_size = 32

train_datagen_mc = ImageDataGenerator(rescale=1./255,
                                   zca_whitening=True,
                                   horizontal_flip=True, 
                                   validation_split=0.2) # set validation split

train_generator_1 = train_datagen_mc.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 

validation_generator_1 = train_datagen_mc.flow_from_directory(
    train_data_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 1474 images belonging to 10 classes.
Found 363 images belonging to 10 classes.


In [9]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze four convolution blocks
for layer in vgg_model.layers[:15]:
    layer.trainable = False

# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)

x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting 
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x) # Softmax for multiclass
transfer_model = Model(inputs=vgg_model.input, outputs=x)

58900480/58889256 [==============================] - 0s 0us/step
0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [10]:
from keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.6,
                              patience=8, 
                              verbose=1, 
                              mode='max', 
                              min_lr=5e-5)
checkpoint = ModelCheckpoint('vgg16_finetune.h15', 
                             monitor= 'val_accuracy', 
                             mode= 'max',
                             save_best_only = True, verbose= 1)

In [11]:
from tensorflow.keras import layers, models, Model, optimizers

In [12]:
#Compile model
learning_rate= 5e-5
nb_epochs = 10
transfer_model.compile(loss="categorical_crossentropy", 
                       optimizer=optimizers.Adam(lr=learning_rate), 
                       metrics=["accuracy"])

#Fit model
history = transfer_model.fit_generator(train_generator_1, 
                                       steps_per_epoch = train_generator_1.samples // batch_size,
                                       validation_data = validation_generator_1, 
                                       validation_steps = validation_generator_1.samples // batch_size,
                                       epochs = nb_epochs, 
                                       shuffle=True, 
                                       callbacks=[lr_reduce],
                                       verbose=1)

Epoch 1/10
46/46 [==============================] - 1075s 24s/step - loss: 2.2104 - accuracy: 0.1921 - val_loss: 1.8920 - val_accuracy: 0.2983
Epoch 2/10
46/46 [==============================] - 1049s 23s/step - loss: 1.6998 - accuracy: 0.3675 - val_loss: 1.4723 - val_accuracy: 0.4659
Epoch 3/10
46/46 [==============================] - 1047s 23s/step - loss: 1.3067 - accuracy: 0.5451 - val_loss: 1.2399 - val_accuracy: 0.5653
Epoch 4/10
46/46 [==============================] - 1048s 23s/step - loss: 1.0550 - accuracy: 0.6214 - val_loss: 1.1648 - val_accuracy: 0.5710
Epoch 5/10
46/46 [==============================] - 1046s 23s/step - loss: 0.7821 - accuracy: 0.7309 - val_loss: 1.2858 - val_accuracy: 0.5625
Epoch 6/10
46/46 [==============================] - 1043s 23s/step - loss: 0.6959 - accuracy: 0.7691 - val_loss: 0.9748 - val_accuracy: 0.6562
Epoch 7/10
46/46 [==============================] - 1062s 23s/step - loss: 0.4912 - accuracy: 0.8329 - val_loss: 0.9639 - val_accuracy: 0.6449

In [13]:
!pip install gradio

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 206 kB 64.0 MB/s 
     |████████████████████████████████| 1.9 MB 43.6 MB/s 
     |████████████████████████████████| 961 kB 58.9 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 3.5 MB 45.9 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=a88b81f2ecb40772b72f4d5ce73ac1c15f175d9f5a7650046d944faa5e06d370
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=8a4dec1239da0d2590a8695e2f4ed7ad7affc9ea9fe88fb6fc418f5c01c20193
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


In [14]:
import gradio
# Load the model
model_dog_breeds = transfer_model

#Load the labels
labels_dog_breeds = breed_list[:10]



In [15]:
#------------------#

# Function for preprocessing an image and predicting the dog breed
def classify_image(image_):
    img = image_.reshape((-1,224, 224, 3))
    prediction = model_dog_breeds.predict(img).flatten()

    return {labels_dog_breeds[i]: float(prediction[i]) for i in range(10)}

# Define the inputs, outputs
image = gradio.inputs.Image(shape=(224,224))
label = gradio.outputs.Label(num_top_classes=3)



In [16]:
#------------------#

# Launch the application
gradio.Interface(
    fn=classify_image,
    inputs=image,
    outputs=label,
    title="Image classification - limited to 10 dog breeds",
).launch(debug=False)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://56769.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://56769.gradio.app')